In [ ]:
import random
import re
from langdetect import detect
import pandas as pd
import configparser
from newspaper import Article, Config
from GoogleNews import GoogleNews
import time
import cohere
import tokenize_uk
from nltk.tokenize import sent_tokenize

config = configparser.ConfigParser()
config.read("keys.config")

hf_key = config['credentials']['hf_key']
key_deepL = config['credentials']['deepl_key']
cohere_key = config['credentials']['cohere']

co = cohere.Client(cohere_key)

In [6]:
PATTERN_MISTAKE = r"Article `download\(\)` failed with 403 Client Error: Forbidden for url: (.*) on URL .*"

#https://github.com/codelucas/newspaper/blob/master/newspaper/configuration.py
def download_newspaper_3k(link, lang, max_retries=5, set_config=False):
    config = Config()
    if set_config:
        config.headers = { #https://iplogger.org/useragents/
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:124.0) Gecko/20100101 Firefox/124.0',
            'Accept-Language': 'uk-UA,uk;q=0.8',
            'Accept': 'test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Referer': 'https://www.google.com/',
            'DNT': '1',
            'Connection': 'keep-alive'
        }
    if link != "Not found":
        for attempt in range(max_retries):
            try:
                config.request_timeout = random.randint(5, 14)
                article = Article(link, language=lang, config=config)
                article.download()
                article.parse()
                article.nlp()
                print("success")
                print(" ".join(article.keywords))
                return article.url, article.text, article.publish_date, article.summary, " ".join(article.keywords)
            except Exception as e:
                print(e)
                if not set_config:
                    match = re.match(PATTERN_MISTAKE, str(e))
                    if match:
                        url = match.group(1)
                        print('Now instead trying with url:', url)
                        url, text, date, summary, keywords = download_newspaper_3k(url, lang, 5, True)
                        return url, text, date, summary, keywords
                print(f"Attempt {attempt+1} of {max_retries}")
                sleep_time = 2 ** attempt + 10
                print(f"Retrying in {sleep_time} seconds.")
                time.sleep(sleep_time)
    else:
        print('Not found')
        return "","","","",""
    print("fail")
    return "","","","",""

In [7]:
def detect_language_langdetect(text):
    try:
        detected_language = detect(text)
        return detected_language
    except Exception as e:
        print("An error occurred:", e)
        return None

In [8]:
def make_war_prompt(text):
    return f'''## Instructions
        Is this text related to/about or somehow mentions a war/armed conflict between Russia and Ukraine? Yes/No? 
        Return answer in the json format: {{"Answer": "Yes"/"No", "Why": <reasoning>}}

        ##Text: 
        
        {text}
        '''

def query_command_r(prompt):
    time.sleep(7)
    try:
        response = co.chat(
            model="command-r",
            message=prompt,
            prompt_truncation="OFF"
        )
        print(response.text)
        return response.text
    except Exception as e:
        print(e)
        return e
    
def query_command_r_documents(documents, prompt):
    #time.sleep(7)
    try:
        response = co.chat(
            message=prompt,
            documents=documents,
            model="command-r",
            temperature=0.3,
            prompt_truncation="OFF"
        )
        print(response.text)
        return response.text
    except Exception as e:
        print(e)
        return e
    
def parse_json(string_json):
    pattern = r'"Answer"\W{,3}:\W{,3}"([^"]+)"\W*"Why"'
    match = re.search(r'"Answer".*:.*"([^"]+)".*"Why"', str(string_json), re.DOTALL)
    if match: 
        return match.group(1)
    else:
        print('no match', string_json)
    return 'Maybe'

In [9]:
def extract_google_news_results(keywords, language='uk', region='UK', start='01/01/2012', end='20/02/2022', top_search=5):
    seconds = random.randint(2, 5)
    time.sleep(seconds)
    googlenews = GoogleNews(lang=language, region=region, start=start,end=end) 
    googlenews.enableException(True)
    try:
        googlenews.get_news(keywords)
        links = googlenews.get_links()
        for i, link in enumerate(links):
            if i == top_search:
                break
            url, text, publish_date, summary, keywords = download_newspaper_3k('http://' + link, language)
            lang = detect_language_langdetect(text)
            if lang == language: #на нужном языке
                print('In right language', lang)
                if_war = query_command_r(make_war_prompt(text))
                if_war_parsed = parse_json(if_war)
                if if_war_parsed == 'No':
                    print('Found a perfect article!')
                    return url, text, publish_date, summary, keywords
                else:
                    print('Article is about war?', if_war_parsed)
                    continue    
            else:
                print('Article is in', lang)
                continue
    except Exception as e:
        print(e)
    print("Not found")
    return "Not found", "", "", "", ""


In [ ]:
sample = pd.read_csv("sources.csv")

In [298]:
sample["ukr_1"] = "Not found"
sample["ukr_2"] = "Not found"

num_keywords = 10
print("Amount of keywords:", num_keywords)
print("Not found news for 1st column:", sample[sample.ukr_1 == "Not found"].shape[0])
print("Not found news for 2nd column:", sample[sample.ukr_2 == "Not found"].shape[0])
sample.loc[sample.ukr_1 == 'Not found', ['ukr_1', 'ukr1_text', 'ukr1_date', 'ukr1_summary', 'ukr1_keywords']] = sample.loc[sample.ukr_1 == "Not found", :].apply(lambda row: 
                                                                  extract_google_news_results(",".join(row.mistral_keywords1_translated.split(';')[:num_keywords])),
                                                                  axis=1, result_type='expand').values
sample.loc[sample.ukr_2 == 'Not found', ['ukr_2', 'ukr2_text', 'ukr2_date', 'ukr2_summary', 'ukr2_keywords']] = sample.loc[sample.ukr_2 == "Not found", :].apply(lambda row: 
                                                                  extract_google_news_results(",".join(row.mistral_keywords2_translated.split(';')[:num_keywords])),
                                                                  axis=1, result_type='expand').values

Amount of keywords: 10
Not found news for 1st column: 236
Not found news for 2nd column: 236
success
орбана європейської голосу реально угорщини європейські відібрати гроші єс ради 7 права єврораді європейського рішення вимагає європарламент право
In right language uk
```json
{
 "Answer": "Yes",
 "Why": "The text mentions that the Hungarian prime minister Viktor Orban blocked the mechanism of Ukraine Facility which prevents Ukraine from receiving 50 billion euros from the EU over the next four years."
}
```
Article is about war? Yes
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
success
євро млрд 202127 коронавірусу роки дуда європейських польща єс європі отримає 160 пандемії насамперед кошти європейського європи бюджету
In right language uk
```json
{
 "Answer": "No",
 "Why": "The text deals with Poland's successful negotiations for a large grant from the EU's new economic recovery fund in the aftermath 

Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
success
росії війна нафтові кремлю скорочення щодо нафти російськосаудівське протистояння мир опек саудівська саудівської завершилося росія вартувало москва аравії
In right language uk
{"Answer": "No", "Why": "The text describes a conflict between Russia and Saudi Arabia caused by a disagreement in the OPEC+ alliance about oil production levels. The conflict resulted in a sharp drop in oil prices, caused by the combination of the COVID-19 crisis and the countries' actions. No mention of a war or armed conflict between Russia and Ukraine is present in the text."}
Found a perfect article!
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not fo

{"Answer": "Yes", "Why": "Text mentions war in the east, which can be referred to the armed conflict between Russia and Ukraine."}
Article is about war? Yes
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
success
ядерної росії україни погрожує перрі атакою україні ядерну й ядерного нато америка відповість зброї сша ядерною росія
In right language uk
```json
{
 "Answer": "Yes",
 "Why": "Text explicitly mentions the war between Russia and Ukraine, as well as Russia's nuclear threats and potential nuclear strikes against Ukraine."
}
```
Article is about war? Yes
Not found
Not found
success
підкреслив вимог тисячі порушення правил осіб безпеки 25 штрафи андрій пожежної оштрафували лісах львівщині гривень
In right language uk
```json
{"Answer": "No", "Why": "The text deals with forest fires and the failure of the Ukrainian authorities to adequately ta

In [ ]:
num_keywords = 7
print("Amount of keywords:", num_keywords)
print("Not found news for 1st column:", sample[sample.ukr_1 == "Not found"].shape[0])
print("Not found news for 2nd column:", sample[sample.ukr_2 == "Not found"].shape[0])
sample.loc[sample.ukr_1 == 'Not found', ['ukr_1', 'ukr1_text', 'ukr1_date', 'ukr1_summary', 'ukr1_keywords']] = sample.loc[sample.ukr_1 == "Not found", :].apply(lambda row: 
                                                                  extract_google_news_results(",".join(row.mistral_keywords1_translated.split(';')[:num_keywords])),
                                                                  axis=1, result_type='expand').values
sample.loc[sample.ukr_2 == 'Not found', ['ukr_2', 'ukr2_text', 'ukr2_date', 'ukr2_summary', 'ukr2_keywords']] = sample.loc[sample.ukr_2 == "Not found", :].apply(lambda row: 
                                                                  extract_google_news_results(",".join(row.mistral_keywords2_translated.split(';')[:num_keywords])),
                                                                  axis=1, result_type='expand').values

In [ ]:
num_keywords = 4
print("Amount of keywords:", num_keywords)
print("Not found news for 1st column:", sample[sample.ukr_1 == "Not found"].shape[0])
print("Not found news for 2nd column:", sample[sample.ukr_2 == "Not found"].shape[0])
sample.loc[sample.ukr_1 == 'Not found', ['ukr_1', 'ukr1_text', 'ukr1_date', 'ukr1_summary', 'ukr1_keywords']] = sample.loc[sample.ukr_1 == "Not found", :].apply(lambda row: 
                                                                  extract_google_news_results(",".join(row.mistral_keywords1_translated.split(';')[:num_keywords])),
                                                                  axis=1, result_type='expand').values
sample.loc[sample.ukr_2 == 'Not found', ['ukr_2', 'ukr2_text', 'ukr2_date', 'ukr2_summary', 'ukr2_keywords']] = sample.loc[sample.ukr_2 == "Not found", :].apply(lambda row: 
                                                                  extract_google_news_results(",".join(row.mistral_keywords2_translated.split(';')[:num_keywords])),
                                                                  axis=1, result_type='expand').values

In [ ]:
num_keywords = 3
print("Amount of keywords:", num_keywords)
print("Not found news for 1st column:", sample[sample.ukr_1 == "Not found"].shape[0])
print("Not found news for 2nd column:", sample[sample.ukr_2 == "Not found"].shape[0])
sample.loc[sample.ukr_1 == 'Not found', ['ukr_1', 'ukr1_text', 'ukr1_date', 'ukr1_summary', 'ukr1_keywords']] = sample.loc[sample.ukr_1 == "Not found", :].apply(lambda row: 
                                                                  extract_google_news_results(",".join(row.mistral_keywords1_translated.split(';')[:num_keywords])),
                                                                  axis=1, result_type='expand').values
sample.loc[sample.ukr_2 == 'Not found', ['ukr_2', 'ukr2_text', 'ukr2_date', 'ukr2_summary', 'ukr2_keywords']] = sample.loc[sample.ukr_2 == "Not found", :].apply(lambda row: 
                                                                  extract_google_news_results(",".join(row.mistral_keywords2_translated.split(';')[:num_keywords])),
                                                                  axis=1, result_type='expand').values

In [304]:
print("Not found news for 1st column:", sample[sample.ukr_1 == "Not found"].shape[0])
print("Not found news for 2nd column:", sample[sample.ukr_2 == "Not found"].shape[0])

Not found news for 1st column: 92
Not found news for 2nd column: 111


In [305]:
ukr_ukr = sample[(sample.ukr1_text != "") & (sample.ukr2_text != "")]
ukr_1 = sample[(sample.ukr1_text != "") & (sample.ukr2_text == "")]
ukr_2 = sample[(sample.ukr1_text == "") & (sample.ukr2_text != "")]

In [310]:
ukr_ukr = ukr_ukr.drop(columns=['text2', 'date2', 'source2', 'extractive_summary2', 'keywords2', 'text1', 'date1', 'source1', 'extractive_summary1', 'keywords1', 'mistral_keywords1', 'mistral_keywords2',
       'mistral_keywords1_translated', 'mistral_keywords2_translated'])
ukr_ukr['url1_lang'] = 'uk'
ukr_ukr['url2_lang'] = 'uk'

new_column_names = {
    'ukr_1': 'source1',
    'ukr1_text': 'text1',
    'ukr1_date': 'date1',
    'ukr1_summary': 'extractive_summary1',
    'ukr1_keywords': 'keywords1',
    'ukr_2': 'source2',
    'ukr2_text': 'text2',
    'ukr2_date': 'date2',
    'ukr2_summary': 'extractive_summary2',
    'ukr2_keywords': 'keywords2',
}
ukr_ukr = ukr_ukr.rename(columns=new_column_names)

In [311]:
ukr_1 = ukr_1.drop(columns=['ukr2_text', 'ukr2_date', 'ukr2_summary', 'ukr2_keywords', 'ukr_2', 'text1', 'date1', 'source1', 'extractive_summary1', 'keywords1', 'mistral_keywords1', 'mistral_keywords2',
       'mistral_keywords1_translated', 'mistral_keywords2_translated'])
ukr_1['url1_lang'] = 'uk'

new_column_names = {
    'ukr_1': 'source1',
    'ukr1_text': 'text1',
    'ukr1_date': 'date1',
    'ukr1_summary': 'extractive_summary1',
    'ukr1_keywords': 'keywords1',
}
ukr_1 = ukr_1.rename(columns=new_column_names)

In [312]:
ukr_2 = ukr_2.drop(columns=['ukr1_text', 'ukr1_date', 'ukr1_summary', 'ukr1_keywords', 'ukr_1', 'text2', 'date2', 'source2', 'extractive_summary2', 'keywords2', 'mistral_keywords2', 'mistral_keywords1',
       'mistral_keywords1_translated', 'mistral_keywords2_translated'])
ukr_2['url2_lang'] = 'uk'

new_column_names = {
    'ukr_2': 'source2',
    'ukr2_text': 'text2',
    'ukr2_date': 'date2',
    'ukr2_summary': 'extractive_summary2',
    'ukr2_keywords': 'keywords2',
}
ukr_2 = ukr_2.rename(columns=new_column_names)

### Ukrainian language summarization

#### Without documents

In [315]:
def make_prompt_ukrainian(news_article):
    return f'''## Інструкція

    Підсумуйте новину, зосередившись на чотирьох основних моментах: 
    1. Хто є головною/другорядною дійовою особою в новинній статті, тобто, хто є предметом історії? 
    2. КОЛИ сталася ситуація, описана в новині? 
    3. ДЕ сталася ситуація, описана в новині? 
    4. ЩО сталося в новині (200-250 слів резюме новини). 
    Відповідь виводити українською мовою.

    ## Приклад виведення
    ХТО: <відповідь на пункт 1 інструкції>
    КОЛИ: <відповідь на пункт 2 інструкції>
    ДЕ: <відповідь на пункт З інструкції>
    ЩО: <відповідь на пункт 4 інструкції (короткий зміст новини обсягом 200-250 слів)>.

    ## Новина

    {news_article}
    '''

In [316]:
ukr_ukr.loc[:, "prompt_summ_text1"] = ukr_ukr.loc[:, "text1"].apply(make_prompt_ukrainian)
ukr_ukr.loc[:, "summ_text1"] = ukr_ukr.loc[:, "prompt_summ_text1"].apply(query_command_r)
ukr_ukr.loc[:, "prompt_summ_text2"] = ukr_ukr.loc[:, "text2"].apply(make_prompt_ukrainian)
ukr_ukr.loc[:, "summ_text2"] = ukr_ukr.loc[:, "prompt_summ_text2"].apply(query_command_r)
ukr_1.loc[:, "prompt_summ_text1"] = ukr_1.loc[:, "text1"].apply(make_prompt_ukrainian)
ukr_1.loc[:, "summ_text1"] = ukr_1.loc[:, "prompt_summ_text1"].apply(query_command_r)
ukr_2.loc[:, "prompt_summ_text2"] = ukr_2.loc[:, "text2"].apply(make_prompt_ukrainian)
ukr_2.loc[:, "summ_text2"] = ukr_2.loc[:, "prompt_summ_text2"].apply(query_command_r)

ХТО: Угорщина та її керівництво на чолі з прем'єр-міністром Віктором Орбаном. 

КОЛИ: Резолюцію щодо сумнівів у спроможності Угорщини головувати в Раді ЄС депутати Європарламенту ухвалили 1 червня 2023 року. 

ДЕ: Європейський парламент та інші інституції ЄС, зокрема, Рада ЄС, Європейська Рада та Єврокомісія. 

ЩО: Євродепутати висловлюють занепокоєння через недостатнє дотримання Угорщиною цінностей та законодавства ЄС, а тому вона не повинна головувати в Раді ЄС з липня 2024 року. Це пов'язано зі систематичними порушеннями Угорщиною принципу верховенства права та прав людини, залежністю судової влади від виконавчої гілки влади, неконкурентним медійним середовищем та іншими проблемами. У вересні 2022 року Європарламент уже приймав аналогічну резолюцію, через що Угорщина була частково позбавлена фінансової допомоги від ЄС. Хоча угорська сторона проводила певні реформи, вони є косметичними та недостатніми. Відсторонити країну від головування в Раді ЄС складно через відсутність такого мех

ХТО: Касем Сулеймані - іранський військовий стратег, якого вбили за наказом президента США Дональда Трампа. 

КОЛИ: 3 січня 2020 року. 

ДЕ: авіаудар, що вбив Сулеймані, відбувся в Багдаді. 

ЩО: Іранського генерала Касема Сулеймані, якого вважають другим за впливовістю політиком Ірану, вбили за наказом президента США. Пентагон підтвердив цю інформацію. У світі по-різному відреагували на цю подію. Президент США Дональд Трамп виправдовує вбивство, стверджуючи, що Сулеймані був причетний до смерті багатьох американців. Держсекретар США Майк Помпео каже, що це було зроблено, аби попередити неминучу атаку на американців у регіоні. Вбивство Сулеймані викликало суперечливі реакції серед американських політиків: демократична частина Конгресу сумнівається в законності такої операції, а республіканці підтримують рішення Трампа. В Ірані обіцяють помститися США за смерть генерала, а Верховний лідер Ірану Алі Хаменеї оголошує в країні триденний траур. Ірак засудив дії США як порушення свого сувере

ХТО: ТОВ «ВАРІАНТИ ЛЬВІВ», а також користувачі сайту https://varianty.lviv.ua, персональні дані яких обробляються компанією.

КОЛИ: Збір персональних даних користувачів відбувається постійно, з моменту реєстрації на сайті та до видалення облікового запису користувача. 

ДЕ: Дії з персональними даними користувачів відбуваються в серверному обладнанні, розташованому за адресою: м.Київ, вул. Максима Кривоноса, 27. 

ЩО: Компанія збирає та обробляє персональні дані користувачів для надання доступу до інтернет-сервісів сайту, обміну інформацією/новинами, відносин у сфері реклами та комунікації, а також для забезпечення виконання вимог українського законодавства. До персональних даних користувачів належать ім'я, прізвище, імейл, номер телефону, дата народження та ін. Доступ до них надається лише уповноваженим працівникам компанії, а також може передаватися третім особам за згодою користувача або у випадках, передбачених законом. 

Кожен користувач має право на доступ до своїх персональних да

ХТО: Україна, Молдова, Боснія і Герцеговина, Грузія, ЄС, Шарль Мішель, Володимир Зеленський, Роберта Мецола, Віктор Орбан

КОЛИ: 14 грудня 2023 року

ДЕ: Брюссель

ЩО: Європейська Рада ухвалила рішення підтримати початок переговорів про вступ України, Молдови та Боснії і Герцеговини до ЄС, як тільки будуть виконані відповідні передумови. Україна виконала рекомендації Єврокомісії, щоб забезпечити позитивне рішення. Це стало можливим після того, як Угорщина, яка раніше блокувала це рішення, отримала від ЄС 10 мільярдів євро на вирішення проблем з верховенством права. Переговори про членство є передостаннім етапом перед приєднанням до ЄС. Раніше Україна отримала статус кандидата. Крім того, очікується затвердження нового 50-мільярдного пакету допомоги Україні від ЄС на наступні чотири роки.
ХТО: Настя Каменських та її мама Лідія Петрівна

КОЛИ: святкування відбулося вчора, а сьогодні Настя Каменських поділилася фотографіями з цієї події

ДЕ: на вечірці у Насті Каменських

ЩО: у будинку На

ХТО: В'ячеслав Шрамович, Панас Маслійник, Віталій Педченко, Олександр Карпюк, Георгій Учайкін та інші.

КОЛИ: 23 березня 2020 року.

ДЕ: Україна.

ЩО: У зв'язку з пандемією коронавірусу в Україні спостерігається високий попит на зброю та боєприпаси. Це явище нагадує ситуацію 2014 року, коли попит на зброю зріс через "параліч" правоохоронної системи та війну на Донбасі. Проте на сьогодні покупці більш за все турбуються про самозахист, тому віддають перевагу доступним моделям зброї. Попит зріс на набої, тоді як на вогнепальну зброю він, навпаки, знизився через призупинення роботи сервісних центрів дозвільної системи під час карантину. Закупівля зброї відбувається також через психологічну причину — люди втрачають довіру до державних структур у скрутні часи та розраховують лише на себе. Магазини зброї закриті на карантин, але деякі з них працюють як пункти видачі інтернет-замовлень. 

Окрема проблема — війна на Донбасі, через яку може виникнути дефіцит боєприпасів для снайперів ЗСУ. Волонт

...

ХТО: Партії "Право і Справедливість", "Громадянська платформа", "Третій шлях", "Нова лівиця" та "Конфедерація", які пройшли до польського Сейму; також згадуються президент Польщі Анджей Дуда та прем'єр-міністр Матеуш Моравецький. 

КОЛИ: Парламентські вибори в Польщі відбулись восени 2023 року — 15 жовтня. 

ДЕ: Вибори проходили в Польщі, тому подія відбувається саме там. 

ЩО: На парламентських виборах в Польщі перемогла партія "Право і Справедливість", але вона втратила більшість у Сеймі, отримавши 196 мандатів. Для порівняння, у попередньому скликанні в неї було 235 місць. Інші партії, які пройшли до Сейму, мають намір створити коаліцію, очолювану "Громадянською платформою" Дональда Туска. Проте, через особливості польського виборчого законодавства та політичні розбіжності між партіями коаліційні переговори обіцяють бути складними. 

Найбільш напруженою є ситуація навколо призначення прем'єр-міністра. Президент Польщі має право висунути свою кандидатуру, але якщо вона не отримає під

ХТО: 49-річний роботодавець та 37-річний громадянин Грузії, його працівник.

КОЛИ: Трагедія сталася 15 квітня.

ДЕ: В одному з виробничих цехів у Гданську.

ЩО: Між 49-річним роботодавцем та його 37-річним працівником, громадянином Грузії, виник конфлікт через заборгованість із заробітної плати. Посперечавшись із іноземцем, роботодавець сів за кермо навантажувача та навмисне наїхав на нього, притиснувши до стіни. Потерпілий помер від отриманих травм. Слідство вважає це вбивством, а не нещасним випадком. Прокуратура інкримінує чоловікові умисне вбивство, той не визнає своєї вини. Суд відправив підозрюваного за ґрати під час слідства.
ХТО: Місто Ковель.

КОЛИ: Реконструкція центральної частини міста та Привокзальної площі планується на наступний рік, а деякі роботи вже розпочаті - наприклад, капітальний ремонт тротуарів на вулиці Незалежності. 

ДЕ: Основна дія відбуватиметься у Ковелі: на вулиці Незалежності, Привокзальній площі, у парку культури і відпочинку імені Лесі Українки та адмі

ХТО: Українські телевізійники - ведучі та працівники телеканалів СТБ, ICTV, 1+1 media, ТЕТ, 2+2, ПлюсПлюс, Суспільне та Еспресо.

КОЛИ: Всесвітній день вишиванки, який відзначають в Україні сьогодні.

ДЕ: Україна. Згадуються Київ, Львів та Дніпро.

ЩО: Українські телеканали по-різному долучилися до відзначення Дня вишиванки. СТБ та ICTV зняли відеоролик з ведучими, які розповідають про традиції вишивання. Співробітники 1+1 media зробили спільне фото та змінили графіку телеканалу. Телеканал ТЕТ підготував проморолик зі святковою графікою. 2+2 поділився сюжетом про походження вишиванки та її символіку. Телеканал ПлюсПлюс привітав глядачів у соцмережах, а Суспільне та Еспресо також долучилися до свята, змінюючи оформлення ефіру та створюючи ролики. Ведучі телеканалів вийшли у вишиванках та розповіли про традиції дарувати вишиванки новонародженим у пологових.
ХТО: Донори (Марк Роуен, Леслі Векснер, Рональд Лаудер, Девід Магерман, Джонатон Якобсон, Джон Хантсман та інші), університети США (

ХТО: Online-видання "Високий Вал", його засновник Олег Головатенко та керівник проекту Артем Євминець.

КОЛИ: Оновлення сайту та запуск мобільної версії відбулись 11 листопада 2018 року. 

ДЕ: Чернігів.

ЩО: Найстаріше онлайн-видання Чернігова "Високий Вал" пройшло оновлення дизайну та функціоналу. Нова команда видання зосередилась на модернізації сайту, зробивши його більш зручним для читачів та адаптивним для мобільних пристроїв. середня відвідуваність val.ua становить близько 10 000 унікальних користувачів щодоби, а спільнота у соцмережі Facebook має 14 000 підписників. "Високий Вал" є одним з найвпливовіших ЗМІ у Чернігівській області. Нині видання відкрите для співпраці з журналістами та медійними структурами, його керівник наголошує на зміні наповнення сайту та запрошує до співпраці.
ХТО: Леонід Брежнєв, маршал Жуков, Віктор Некрасов, Сталін, російські солдати, українські солдати, радянське керівництво, Путін. 

КОЛИ: 9 травня 1981 року. Описувані події пов'язані з Другою світово

ХТО: Громадська спілка «Українська мережа за права дитини» (УМПД) та її голова правління Дар'я Касьянова.

КОЛИ: Дорожня карта була розроблена у 2023 році та презентована 15 квітня.

ДЕ: Події відбуваються в Києві, де розташована пресслужба УМПД.

ЩО: УМПД презентувала Дорожню карту у сфері забезпечення прав та найкращих інтересів дитини, яка складається з 10 розділів та охоплює широкий спектр питань, пов'язаних з правами дитини. Кожен розділ містить рекомендації для органів влади щодо вирішення проблем у відповідних сферах, таких як культура сприйняття дитини в суспільстві, законодавство та політики, урядове управління, людські та фінансові ресурси, послуги для дітей та сімей, догляд та захист дітей. Дорожня карта створена на основі міжнародних принципів та стратегій у сфері прав дитини та є компасом для громадського сектору та влади у їхній діяльності. Презентація документа відбулася в рамках проєкту «Поверни Україну для всіх дітей» за участі представників влади та громадськості. Мет

#### With documents

In [3]:
def generate_documents_command_r(text):
    documents = []
    for i, paragraph in enumerate(tokenize_uk.tokenize_uk.tokenize_text(text)):
        paragraph_joined = ""
        for sentence in paragraph:
            paragraph_joined += " ".join(sentence)
        documents.append({"title": f'Paragraph {i}', "snippet": paragraph_joined})
    return documents

In [318]:
ukr_ukr.loc[:, "documents_text1"] = ukr_ukr.loc[:, "text1"].apply(generate_documents_command_r)
ukr_ukr.loc[:, "documents_text2"] = ukr_ukr.loc[:, "text2"].apply(generate_documents_command_r)
ukr_1.loc[:, "documents_text1"] = ukr_1.loc[:, "text1"].apply(generate_documents_command_r)
ukr_2.loc[:, "documents_text2"] = ukr_2.loc[:, "text2"].apply(generate_documents_command_r)

In [25]:
prompt_documents_ukr = '''## Інструкція

    Підсумуйте новину в документах, що додаються, зосередившись на чотирьох основних моментах: 
    1. Хто є головною/другорядною дійовою особою в новинній статті, тобто, хто є предметом історії? 
    2. КОЛИ сталася ситуація, описана в новині? 
    3. ДЕ сталася ситуація, описана в новині? 
    4. ЩО сталося в новині (200-250 слів резюме новини). 
    Відповідь виводити українською мовою.

    ## Приклад виведення
    ХТО: <відповідь на пункт 1 інструкції>
    КОЛИ: <відповідь на пункт 2 інструкції>
    ДЕ: <відповідь на пункт З інструкції>
    ЩО: <відповідь на пункт 4 інструкції (короткий зміст новини обсягом 200-250 слів)>.
    '''

In [320]:
ukr_ukr.loc[:, "summ_with_documents_text1"] = ukr_ukr.loc[:, "documents_text1"].apply(query_command_r_documents, args=(prompt_documents_ukr,))
ukr_ukr.loc[:, "summ_with_documents_text2"] = ukr_ukr.loc[:, "documents_text2"].apply(query_command_r_documents, args=(prompt_documents_ukr,))
ukr_1.loc[:, "summ_with_documents_text1"] = ukr_1.loc[:, "documents_text1"].apply(query_command_r_documents, args=(prompt_documents_ukr,))
ukr_2.loc[:, "summ_with_documents_text2"] = ukr_2.loc[:, "documents_text2"].apply(query_command_r_documents, args=(prompt_documents_ukr,))

ХТО: Угорщина, Михайло Драпак — директор Програми європейських студій Ради зовнішньої політики «Українська призма».

КОЛИ: Угорщина мала б отримати статус країни, що головує в Раді ЄС в липні 2024 року, але може не встигати підготуватися через політичні зміни в ЄС, що заплановані на той же період.

ДЕ: primarily в Угорщині, також згадуються Іспанія, Бельгія та інші країни ЄС. 

ЩО: Європейський парламент сумнівається, що Угорщина зможе виконувати обов'язки країни, що головує в Раді ЄС, через проблеми з дотриманням законодавства ЄС та принципів співпраці. Це вже не перше зауваження на адресу Угорщини від ЄС, тож парламент закликає інші інституції ЄС вплинути на ситуацію в країні. Угорщину можуть примусити виконати певні вимоги, але це складний політично процес, тож ймовірніше, що країна не буде відсторонена від участі в Раді. Проте, за 13 місяців знайти заміну Угорщині буде майже неможливо. Тож Іспанія та Бельгія можуть поділити між собою її обов'язки.
ХТО: Наталія Гончаренко – завідува

ХТО: Головною дійовою особою є субваріант коронавірусу "Дженні", який був виявлений в Україні. Також згадується Ігор Кузін - головний санітарний лікар, який повідомляв про середній або легкий перебіг захворювання.

КОЛИ: Згадується два періоди часу - січень 2024 року, коли були виявлені перші випадки захворювання, та епідсезон 2023-2024 років, коли домінував субваріант "Дженні".

6 лютого повідомлялося про зростання захворюваності на грип та ГРВІ серед дітей.

ДЕ: Виявлення субваріанту "Дженні" відбулося в 12 областях України, серед них: Житомирська, Івано-Франківська, Київська, Львівська, Одеська, Сумська, Хмельницька, Волинська, Черкаська, Чернівецька, Рівненська та Чернігівська області.
Особливо відзначається Київ - столиця України.

ЩО: В Україні протягом січня 2024 року було виявлено 42 випадки захворювання субваріантом Omicron "Дженні". Хвороба протікає у легкій або середньотяжкій формі. Майже половина хворих - люди старші 60 років, більшість хворих не вакциновані або не завершил

ХТО: Основною дійовою особою є Греція, яка будує стіну на кордоні з Туреччиною, щоб зупинити потік мігрантів з Афганістану. Другорядними діячами є мігранти/біженці з Афганістану, міністр захисту громадян Греції Михаліс Хрисохоїдіс, прем'єр-міністр Греції Кіріакос Міцотакіс та президент Туреччини Реджеп Тайіп Ердоган.

КОЛИ: Будівництво стіни почалося 21 серпня 2021 року.

ДЕ: Дія відбувається на кордоні Греції та Туреччини.

ЩО: Греція будує 40-кілометрову стіну та встановлює систему відеоспостереження на своєму кордоні з Туреччиною, щоб зупинити приплив мігрантів з Афганістану. Це пов'язано з погіршенням ситуації в Афганістані після захоплення країни талібами, через що багато афганців тікають зі своєї країни. Туреччина закликала європейські країни прийняти мігрантів, через що Греція посилює свій кордон. Це нагадує міграційну кризу 2015 року, коли Греція також була воротами для мігрантів до ЄС. Тоді понад мільйон людей пройшли через Грецію до інших країн Європи, а зараз грецька влада п

ХТО: Текст має на увазі користувача або суб'єкта персональних даних та Компанію або сайт https://varianty.lviv.ua, який належить ТОВ "ВАРІАНТИ ЛЬВІВ". 

КОЛИ: Час дії не вказано. Проте, у пункті 23 йдеться про те, що персональні дані зберігаються не більш як це необхідно, відповідно до мети їх обробки, а після того, як суб'єкт персональних даних припиняє бути користувачем сайту, його дані також автоматично видаляються.

ДЕ: подія відбувається в Україні. Адреса розташування серверів, де зберігаються дані, - м. Київ, вул. Максима Кривоноса, 27.

ЩО: Сайт https://varianty.lviv.ua, який надається ТОВ "ВАРІАНТИ ЛЬВІВ", збирає персональні дані користувачів, які ті добровільно надають при реєстрації або користуванні сервісами сайту. Це, наприклад, ім'я та прізвище, логін та пароль доступу, адреса електронної пошти, контактний номер телефону, дата народження та ін. Компанія зобов'язується захищати цю конфіденційну інформацію та не поширювати її у відкритому доступі. Окрім того, сайт може викор

ХТО: Генерал Касем Сулеймані, другою за впливовістю особою в Ірані, та президент США Дональд Трамп, який віддав наказ про його вбивство.

КОЛИ: 3 січня 2020 року.

ДЕ: Іран та Ірак. Основна подія статті - вбивство Касема Сулеймані, яке відбулось біля аеропорту Багдада (Ірак).

ЩО: Генерал Касем Сулеймані, який був командувачем спецпідрозділу Корпусу вартових ісламської революції Ірану - Сил Кудс, був убитий за наказом президента США Дональда Трампа. Пентагон провів успішну операцію з його ліквідації через підозри у плануванні нападів на американських дипломатів та службовців в Іраку. Сулеймані вважали відповідальним за смерть багатьох американців та поранення ще більшої їх кількості під час різних атак та битв в Іраку, Сирії та інших країнах Близького Сходу, де він збільшував вплив Ірану, підтримуючи проіранські угруповання, що боролися проти США та Ізраїлю. Його смерть може спричинити зростання напруги між Іраном та США в Іраку і Перській затоці, а також зростання цін на нафту. Іран о

...

ХТО: Марк Рютте, прем'єр-міністр Нідерландів, який стає новим генеральним секретарем НАТО, замінивши чинного генсека Єнса Столтенберга. 

КОЛИ: Єнс Столтенберг піде у відставку в жовтні 2024 року, а Марк Рютте приступить до виконання своїх обов'язків з 1 жовтня того ж року.

ДЕ: Північноатлантична рада затвердила призначення Марка Рютте на посаду генерального секретаря НАТО.

ЩО: Чинний генеральний секретар НАТО Єнс Столтенберг, який перебуває на цій посаді 10 років, вирішує піти у відставку. За рішеннями Північноатлантичної ради, його замінить прем'єр-міністр Нідерландів Марк Рютте. Після того, як з перегонів за цю посаду вийшов президент Румунії Клаус Йоганніс, Рютте залишився єдиним кандидатом, якого підтримали лідери всіх країн НАТО.
ХТО: Головними акторами у новині є інвестори у США, які побоюються другої хвилі пандемії COVID-19 та її наслідків для економіки країни.

КОЛИ: Ключові фондові індекси США різко впали 11 червня.

ДЕ: Події відбуваються на Нью-Йоркській фондовій біржі та

ХТО: 49-річний роботодавець та 37-річний працівник підприємства, громадянин Грузії.

КОЛИ: 15 квітня.

ДЕ: Виробничий цех у Гданську.

ЩО: Під час конфлікту через невиплачену заробітну плату 37-річний працівник отримав смертельні травми від навантажувача, за кермом якого був його 49-річний роботодавець. За даними слідства, це було умисне вбивство, а не нещасний випадок. На відео з камер спостереження видно, що поведінка водія була свідомою та цілеспрямованою. Прокуратура висунула обвинувачення роботодавцю у вчиненні вбивства, але той не визнає своєї вини. Суд відправив підозрюваного за ґрати на час слідства.
ХТО: Головними діячами є ковельчани, міська рада, архітектори з Луцька Дмитро Кирилюк, Вікторія Кирилюк та Оксана Костюк, а також мер міста Ігор Чайка. 

КОЛИ: Концепція реконструкції центральної частини міста та Привокзальної площі була представлена ще влітку, а підписання договору щодо розробки проєктно-кошторисної документації відбулось нещодавно. 

ДЕ: подія відбувається у міст

ХТО: Ведучі українських телеканалів: СТБ, ICTV, «1+1 media», "Сніданок з 1+1", Телеканал ТЕТ, "2+2", "ПлюсПлюс", "Еспресо".

КОЛИ: Сьогодні в Україні відзначається Всесвітній день вишиванки.

ДЕ: Україна. Ведучі українських телеканалів вийшли в ефір або опублікували дописи в соцмережах на тему вишиванок.

ЩО: У День вишиванки ведучі українських телеканалів взяли участь у різних акціях. Ведучі СТБ та ICTV знялись у відеоролику, де розповідають цікаві факти про традиції вишивання в Україні. Телеканал «1+1 media» зробив традиційне спільне фото на сходах свого офісу, а ведучі програми "Сніданок з 1+1" розповіли про традицію дарувати вишиванки новонародженим у пологових Києва, Львова та Дніпра. Телеканал ТЕТ підготував проморолик, у якому закликав українців одягати вишиванки — символ єдності, а також змінив логотип у соцмережах. Телеканал "2+2" у соцмережах опублікував сюжет про походження вишиванки та її сакральне значення, а "ПлюсПлюс" привітав свою аудиторію з Днем вишиванки та опублікув

ХТО: Windows

КОЛИ: Зараз

ДЕ: На комп'ютерах

ЩО: Продуктивність комп'ютера може постраждати через ресурсоємні браузери. Тому користувачам низькопродуктивних комп'ютерів рекомендується вибирати легші браузери, аби не негативно впливати на продуктивність. У новині перераховано сім таких легких браузерів, які не завантажують систему й допомагають поліпшити її продуктивність. Це може бути хорошим рішенням для користувачів, які хочуть безпечного та легкого в використанні браузера.
ХТО: Головною дійовою особою є команда онлайн-видання "Високий Вал", зокрема керівник проекту Артем Євминець та головний редактор Юрій Михуля.

КОЛИ: Оновлення сайту відбулося 11 листопада 2018 року.

ДЕ: подія відбулася в  Чернігові.

ЩО: Найстаріше онлайн-видання Чернігова "Високий Вал", засноване у 2001 році, undergo оновлення 11 листопада 2018 року: був змінений дизайн сайту та запущена його мобільна версія, що зробило його більш зручним для читачів. Разом з тим було змінено і наповнення ресурсу, а команда в

ХТО: Головною дійовою особою є громадська спілка «Українська мережа за права дитини» (УМПД), її голова правління Дар'я Касьянова та програмна директорка МБО «БФ “СОС Дитячі Містечка” Україна». Іншими ключовими особами є експерти УМПД, представниця Координаційного центру з розвитку сімейного виховання та догляду за дітьми, юристка Регіонального центру прав людини Катерина Рашевська та діти й батьки, які були залучені до розробки Дитячої дорожньої карти.

КОЛИ: Презентація Дитячої дорожньої карти відбулася 15 квітня 2023 року, а сама Дорожня карта була розроблена протягом 2023 року. 

ДЕ: Подія відбулася в Києві.

ЩО: «Українська мережа за права дитини» презентувала Дитячу дорожню карту, яка спрямована на забезпечення прав та найкращих інтересів дитини. Дорожня карта складається з 10 розділів та 10 компонентів, які охоплюють широкий спектр питань, пов'язаних з правами дитини, включаючи культуру сприйняття дитини в суспільстві, законодавство та політику, управління системою, дані та аналі

## Other languages summarization

In [53]:
def other_languages_generate_documents_command_r_version_1(text, language='russian'):
    documents = []
    total_sentences_paragraph = 0
    summed_paragraph = ""
    i = 1
    for paragraph in text.split('\n'):
        sentences = sent_tokenize(paragraph, language=language)
        sent_in_paragraph = len(sentences)
        if (total_sentences_paragraph + sent_in_paragraph) <= 10: #too small paragraph
            summed_paragraph += ('\n' + paragraph)
            total_sentences_paragraph += sent_in_paragraph
        elif sent_in_paragraph <= 10:
            documents.append({"title": f'Paragraph {i}', "snippet": summed_paragraph})
            i += 1
            total_sentences_paragraph = sent_in_paragraph
            summed_paragraph = paragraph 
        else:
            if total_sentences_paragraph > 0:
                documents.append({"title": f'Paragraph {i}', "snippet": summed_paragraph})
                i += 1
                summed_paragraph = ""
                total_sentences_paragraph = 0
            j = 0
            summed_sentences = ""
            for sentence in sentences:
                if j < 10:
                    summed_sentences += (" " + sentence)
                    j += 1
                else:
                    documents.append({"title": f'Paragraph {i}', "snippet": summed_sentences})
                    i += 1
                    summed_sentences = ""
                    j = 0
            if j > 0:    
                documents.append({"title": f'Paragraph {i}', "snippet": summed_sentences})
                i += 1
    if total_sentences_paragraph > 0:      
        documents.append({"title": f'Paragraph {i}', "snippet": summed_paragraph})
    return documents

### Russian language summarization

#### Without documents

In [322]:
def make_prompt_russian(news_article):
    return f'''## Инструкция

    Подытожите новость, сосредоточившись на четырех основных моментах: 
    1. КТО является главным/второстепенным действующим лицом в новости? 
    2. КОГДА произошла ситуация, описанная в новости? 
    3. ГДЕ произошла ситуация, описанная в новости? 
    4. ЧТО произошло в новости? (200-250 слов суммаризация/краткий пересказ новости) 
    
    Ответ должен быть на русском языке.

    ## Пример ответа
    КТО: <ответ на 1ый пункт инструкции>
    КОГДА: <ответ на 2й пункт инструкции>
    ГДЕ: <ответ на 3й пункт инструкции>
    ЧТО: <ответ на 4ый пункт инструкции (краткое содержание новости объемом 200-250 слов)>.

    ## Новость

    {news_article}
    '''

In [323]:
mixed_first_part_ru = ukr_1[ukr_1.url2_lang == 'ru'].copy()
mixed_first_part_ru.loc[:, "prompt_summ_text2"] = mixed_first_part_ru.loc[:, "text2"].apply(make_prompt_russian).values
mixed_first_part_ru.loc[:, "summ_text2"] = mixed_first_part_ru.loc[:, "prompt_summ_text2"].apply(query_command_r).values
mixed_second_part_ru = ukr_2[ukr_2.url1_lang == 'ru'].copy()
mixed_second_part_ru.loc[:, "prompt_summ_text1"] = mixed_second_part_ru.loc[:, "text1"].apply(make_prompt_russian).values
mixed_second_part_ru.loc[:, "summ_text1"] = mixed_second_part_ru.loc[:, "prompt_summ_text1"].apply(query_command_r).values

КТО: Дорожная полиция Армении и водители общественного транспорта.
КОГДА: Вторник, 31 марта 2020 года.
ГДЕ: Армения.
ЧТО: Дорожная полиция Армении продолжает контролировать выполнение решения Коменданта о режиме изоляции и ограничении свободного передвижения в связи с пандемией. За превышение допустимого числа пассажиров в общественном транспорте было составлено около 60 протоколов об административных нарушениях. Пассажиры из таких транспортных средств были высажены. Полиция призывает водителей соблюдать установленные правила и обещает продолжать проверки. Также правоохранительные органы рекомендуют пассажирам отказываться от поездок в переполненных автобусах и маршрутных такси, чтобы не подвергать себя опасности.
КТО: Владимир Зеленский - основной; Георгий Мурадов - второстепенный. 

КОГДА: Ситуация развивалась в период после аннексии Крыма Россией, точная дата не указана. 

ГДЕ: Украина и Крым. 

ЧТО: Владимир Зеленский, президент Украины, заявил в интервью, что россияне через нескол

КТО: Главными действующими лицами являются Альфа-банк и его клиенты, чьи карты были заблокированы.

КОГДА: Новость произошла 7 марта.

ГДЕ: В основном действие происходит в Москве, также упоминается конец декабря 2019 года.

ЧТО: Альфа-банк заблокировал карты своих клиентов из-за возможной компрометации их номеров. Это вызвало панику среди клиентов, горячая линия банка была перегружена, а в чате не отвечали на запросы. Впоследствии банк заявил, что угроза была ложной и карты были разблокированы. Однако для обеспечения безопасности банк планирует бесплатно перевыпустить все заблокированные карты. Это уже не первый подобный случай с картами Альфа-банка — в конце декабря 2019 года был ещё один крупный сбой.
КТО: Банк России

КОГДА: 6 апреля 2020 года

ГДЕ: Москва

ЧТО: Банк России объявил о проведении аукциона репо "тонкой настройки" на 500 млрд рублей. Первая часть сделок будет исполнена сразу после аукциона, 6 апреля, вторая часть - 8 апреля. ЦБ также заявил, что текущий уровень ликвидн

КТО: Предприятия и жители Московской области, ТАСС, губернатор и правительство региона, компании, которые организовали производство медицинских масок.

КОГДА: Ситуация развивается на фоне пандемии COVID-19, о которой ВОЗ объявила 11 марта. В Московской области вводят "масочный режим", начиная с 12 мая. 

ГДЕ: Основная новость происходит в Московской области, Россия. Также упоминаются город Ухань, где началась пандемия, и штаб-квартира ВОЗ. 

ЧТО: В сутки предприятия Московской области производят более 600 тысяч медицинских масок, что в два раза больше, чем было ранее. Это связано с введением в регионе "масочного режима" из-за пандемии COVID-19. 14 предприятий в Подмосковье уже вовлечены в производство масок, ещё девять компаний готовятся к перепрофилированию. Помимо этого, власти принимают меры по контролю над соблюдением нового режима: ношение масок в общественном транспорте будет контролироваться сотрудниками полиции.
КТО: Главными действующими лицами являются жители Украины, которые

КТО: Правительство Азербайджана
КОГДА: С 24 марта по 20 апреля 2020 года
ГДЕ: По всей территории Азербайджана
ЧТО: В стране вводится карантинный режим из-за пандемии коронавируса. Граждане старше 65 лет должны оставаться дома, приостанавливаются межгородские перевозки, ограничивается въезд и выезд из Баку и ещё нескольких районов. В столице рекомендуется не пользоваться метро и предоставлен список мест, где можно собираться группами (не более 10 человек). Контроль за соблюдением режима возложен на МВД. 

Азербайджан вводит строгие меры на фоне пандемии COVID-19, целью является сдерживание распространения вируса и снижение рисков для здоровья и жизни граждан.
КТО: США  

КОГДА: 18 июня 2020 года 

ГДЕ: Черное море 

ЧТО: Десантный корабль-док ВМС США Oak Hill направился в Черное море для совместного патрулирования с эсминцем США Porter. Это уже четвёртый корабль ВМС США, зашедший в Черное море в 2020 году. Силы Южного военного округа России контролируют ситуацию.
КТО: Гости Тверской ули

#### With documents

In [324]:
prompt_command_r_ru = '''
## Инструкция

    Подытожьте новость в прилагаемых документах, сосредоточившись на четырех основных моментах: 
    1. КТО является главным/второстепенным действующим лицом в новости? 
    2. КОГДА произошла ситуация, описанная в новости? 
    3. ГДЕ произошла ситуация, описанная в новости? 
    4. ЧТО произошло в новости? (100-250 слов суммаризация/краткий пересказ новости). 
    Ответ должен быть на русском языке.

## Пример ответа

    КТО: <ответ на 1ый пункт инструкции>
    КОГДА: <ответ на 2й пункт инструкции>
    ГДЕ: <ответ на 3й пункт инструкции>
    ЧТО: <ответ на 4ый пункт инструкции (краткое содержание новости объемом 200-250 слов)>.
    '''

In [325]:
mixed_first_part_ru.loc[:, "documents_text2"] = mixed_first_part_ru.loc[:, "text2"].apply(other_languages_generate_documents_command_r_version_1, language='russian')
mixed_second_part_ru.loc[:, "documents_text1"] = mixed_second_part_ru.loc[:, "text1"].apply(other_languages_generate_documents_command_r_version_1, language='russian')
mixed_first_part_ru.loc[:, "summ_with_documents_text2"] = mixed_first_part_ru.loc[:, "documents_text2"].apply(query_command_r_documents, args=(prompt_command_r_ru, ))
mixed_second_part_ru.loc[:, "summ_with_documents_text1"] = mixed_second_part_ru.loc[:, "documents_text1"].apply(query_command_r_documents, args=(prompt_command_r_ru, ))

КТО: Дорожная полиция РА, которая контролирует и проверяет выполнение решения Коменданта о режиме изоляции и ограничении свободного передвижения; водители общественного транспорта, которые нарушили правила и перевозили слишком много пассажиров.

КОГДА: Вторник, 31 марта 2020 года.

ГДЕ: Республика Армения.

ЧТО: С момента вступления в силу решения Коменданта о режиме изоляции было составлено около 60 протоколов об административных нарушениях в отношении водителей общественного транспорта. Полиция высаживала лишних пассажиров из автобусов и микроавтобусов. Правоохранительные органы призывают граждан Армении не пользоваться переполненными автобусами и микроавтобусами из соображений безопасности.
КТО: Главными действующими лицами являются президент Украины Владимир Зеленский и вице-премьер крымского правительства Георгий Мурадов.

КОГДА: Комментарий Владимира Зеленского, на который реагирует Георгий Мурадов, был сделан в настоятельное время.

ГДЕ: Ситуация происходит между Украиной и Росс

КТО: Главными действующими лицами являются Альфа-банк и его клиенты.

КОГДА: Массовая блокировка карт произошла 7 марта, ранее в конце декабря 2019 года у банка уже был крупный сбой с картами.

ГДЕ: Ситуация произошла в Москве.

ЧТО: Альфа-банк блокировал карты своих клиентов из-за возможной компрометации их номеров, в результате чего карты оказались недоступны для использования. Однако, после того как банк убедился, что средств клиентов ничто не угрожает, блокировка карт была прекращена. Later, the bank plans to re-issue the cards for free to ensure customers' funds are completely safe. The incident resulted in a large number of customer queries, overwhelming the bank's hotline.
КТО: Банк России, ЦБ

КОГДА: 6 апреля 2020 года

ГДЕ: Москва

ЧТО: Банк России принял решение провести аукцион репо "тонкой настройки" на сумму 500 млрд рублей. Первая часть сделок будет выполнена 6 апреля, а вторая - 8 апреля. Это сделано для стабилизации финансовой системы, поскольку на фоне ослабления рубля

КТО: Владимир Путин – президент Российской Федерации, который выступил против появления родителей №1 и №2; также упоминается рабочая группа по подготовке конституционных поправок, члены Госдумы и эксперты, полагающие, что Путин хочет остаться у власти.

КОГДА: Встреча президента с рабочей группой по конституционным поправкам состоялась 21 января.

ГДЕ: Встреча, на которой было сделано это заявление, прошла в России, вероятно, в Москве – месте нахождения Госдумы и президента РФ.

ЧТО: Владимир Путин решительно заявил, что Россия при нём не будет признавать понятия "родитель №1" и "родитель №2". По его мнению, в стране должны оставаться традиционные понятия "папа" и "мама". Это заявление было сделано на встрече с рабочей группой, которой поручено подготовить поправки к конституции. Путин подчеркнул, что защита традиционной семьи является правильной идеей. Также есть предположения, что Путин хочет остаться у власти, поэтому и вносит соответствующие изменения в основной закон страны.
КТО: 

КТО: Канцлер Австрии Себастьян Курц, канцлер Германии Ангела Меркель и президент Франции Эмманюэль Макрон.

КОГДА: План канцлера Меркель и президента Макрона был представлен 18 мая, альтернативный план был разработан на следующий день — 19 мая.

ГДЕ: Ситуация произошла в Австрии, Германии и Франции.

ЧТО: Канцлер Германии Ангела Меркель и президент Франции Эмманюэль Макрон предложили план восстановления европейской экономики после кризиса, вызванного пандемией коронавируса. Он предусматривает создание фонда объёмом 500 млрд евро, помогающего пострадавшим от кризиса странам при посредстве многолетнего финансового плана ЕС. Австрия, Дания, Нидерланды и Швеция предложили свой план, согласно которому странам-членам ЕС будет предоставлена разовая экономическая помощь в виде двухлетних выгодных кредитов. Эти государства выступают против увеличения размера общеевропейского бюджета и обобществления долгов.
КТО: Заместитель министра обороны России генерал-лейтенант Юнус-Бек Евкуров

КОГДА: Подг

### English language summarization

#### Without documents

In [326]:
def make_prompt_english(news_article):
    return f'''## Instructions

    Summarize a news article focusing on four main points: 
    1. WHO is the main/secondary actor in the news article? 
    2. WHEN did the situation described in the news article occur? 
    3. WHERE did the situation described in the news article take place? 
    4. WHAT happened in the news article? (200-250 words summarization of the news article)

    ## Example output
    WHO: <answer to the 1st point of the Instructions>
    WHEN: <answer to the 2nd point of the Instructions>
    WHERE: <answer to the 3rd point of the Instructions>
    WHAT: <answer to the 4th point of the Instructions (200-250 words summary of the news article)>.

    ## News Article

    {news_article}
    '''

In [327]:
mixed_first_part_en = ukr_1[ukr_1.url2_lang == 'en'].copy()
mixed_first_part_en.loc[:, "prompt_summ_text2"] = mixed_first_part_en.loc[:, "text2"].apply(make_prompt_english).values
mixed_first_part_en.loc[:, "summ_text2"] = mixed_first_part_en.loc[:, "prompt_summ_text2"].apply(query_command_r).values
mixed_second_part_en = ukr_2[ukr_2.url1_lang == 'en'].copy()
mixed_second_part_en.loc[:, "prompt_summ_text1"] = mixed_second_part_en.loc[:, "text1"].apply(make_prompt_english).values
mixed_second_part_en.loc[:, "summ_text1"] = mixed_second_part_en.loc[:, "prompt_summ_text1"].apply(query_command_r).values

WHO: French President Emmanuel Macron is the primary actor of the article, with Polish President Andrzej Duda and Prime Minister Mateusz Morawiecki as secondary actors, representing the receiving end of Macron's messages.

WHEN: The situation unfolded on Monday, during Macron's first visit to Poland since his election as President.

WHERE: The article primarily discusses events in Warsaw, Poland.

WHAT: Emmanuel Macron attempted to reassure Poland of France's commitment to European security, explaining his nation's stance as pro-European rather than taking sides in Russo-Polish tensions. During his visit, Macron emphasized the need for European firmness towards Russia and sought to convince the Polish leadership that their security was best protected by European alignment. He urged Poland to trust Europe's collective defense capabilities, believing it to be a more reliable deterrent against potential Russian aggression than any other alternative. 

Macron's message was one of unity, ai

WHO: Blue and White Party leader Benny Gantz and Health Minister Yaakov Litzman

WHEN: The events occurred on Sunday night, with Gantz making a speech denouncing the government's handling of the health system. 

WHERE: The situation took place in Israel, with Gantz delivering his speech in the mixed Muslim, Druze, and Christian Arab local council of Abu Snan, located northeast of Acre.

WHAT: Gantz strongly criticized the Israeli health system, highlighting the issue of thousands of people dying from hospital-acquired infections and deficiencies in healthcare. He pledged to improve the system by funding 2,500 hospital beds, creating positions for healthcare professionals, and using royalties from natural gas reserves to invest in healthcare. Additionally, he promised to amend the Nation State Law, which has been criticized for making Druze citizens feel like second-class residents. 

Meanwhile, Health Minister Yaakov Litzman sent a letter to Gantz, outlining the steps being taken by th

WHO: The main actor in the news article is the Governor of Wisconsin, Tony Evers. He is a key figure among the state's Democrats and his actions, such as vetoes, are currently hard to override due to the lack of Republican members in the Legislature. 

WHEN: The article does not specify an exact date or time period for the situations described. However, the events seem to be ongoing as they relate to the current political climate and upcoming elections in Wisconsin. 

WHERE: The article primarily discusses political events in the state of Wisconsin, with specific reference to the Governor's interactions with the Legislature and the upcoming elections. 

WHAT: Governor Evers is concerned about the potential contamination of nitrates and PFAS chemicals in the state's water sources. He criticizes the downplaying of this issue by others and fears overregulation may be stunted due to political pressures. 

Evers also stresses the importance of the upcoming elections, urging candidates to vi

WHO: The main actor in this news article seems to be the Wayback Machine, a service provided by Archive.org. It is a digital time capsule that allows users to visit previous versions of websites and explore the history of the internet.

WHEN: The article does not specify a particular date or timeframe, but it appears to be relevant as of June 2024, as that is the current date mentioned in the article.

WHERE: The Wayback Machine service is located on the internet and accessible to users worldwide.

WHAT: The news article discusses the Wayback Machine's ability to transport users back in time on the web. It's a valuable tool for researchers, historians, and anyone curious about the evolution of the internet. The article emphasizes the machine's uniqueness in capturing not only the content but also the context of historical web pages, including design elements and interactive features.

It's also mentioned that the Wayback Machine has archived more than 430 billion web pages, providing a

WHO: The main actor in this news article is India's e-commerce industry, which is worth over $50 billion. However, the country's lack of a comprehensive ecommerce policy creates regulatory confusion. Separate guidelines drafted by different ministries add to the industry's woes.

WHEN: The situation described in the article appears to be an ongoing issue, as the most recent development mentioned is the draft of rules by the Ministry for Electronics, Telecom and IT, which occurred in June. However, the upheaval started months ago with the implementation of new Foreign Direct Investment (FDI) rules. 

WHERE: The article focuses on India's efforts to regulate its e-commerce industry, primarily the impact of the new rules on the country's leading online marketplaces.

WHAT: India's lucrative e-commerce industry is in a state of flux due to a lack of clear regulation. A recent policy shift has banned FDI-funded marketplaces from selling their own goods directly and instead, forces them to u

#### With documents

In [328]:
prompt_command_r_en = '''
## Instructions

    Summarize a news article from the included documents focusing on four main points: 
    1. WHO is the main/secondary actor in the news article? 
    2. WHEN did the situation described in the news article occur? 
    3. WHERE did the situation described in the news article take place? 
    4. WHAT happened in the news article? (200-250 words summarization of the news article)

## Example output
    WHO: <answer to the 1st point of the Instructions>
    WHEN: <answer to the 2nd point of the Instructions>
    WHERE: <answer to the 3rd point of the Instructions>
    WHAT: <answer to the 4th point of the Instructions (200-250 words summary of the news article)>.

## News Article
    '''

In [329]:
mixed_first_part_en.loc[:, "documents_text2"] = mixed_first_part_en.loc[:, "text2"].apply(other_languages_generate_documents_command_r_version_1, language='english')
mixed_second_part_en.loc[:, "documents_text1"] = mixed_second_part_en.loc[:, "text1"].apply(other_languages_generate_documents_command_r_version_1, language='english')
mixed_first_part_en.loc[:, "summ_with_documents_text2"] = mixed_first_part_en.loc[:, "documents_text2"].apply(query_command_r_documents, args=(prompt_command_r_en, ))
mixed_second_part_en.loc[:, "summ_with_documents_text1"] = mixed_second_part_en.loc[:, "documents_text1"].apply(query_command_r_documents, args=(prompt_command_r_en, ))

WHO: The main actor in this news article is French President Emmanuel Macron, who visited Poland and spoke to Polish President Andrzej Duda and Prime Minister Mateusz Morawiecki. 

WHEN: Emmanuel Macron's visit to Poland occurred on a Monday while he was still serving as president, but the article does not specify a specific date.

WHERE: The situation took place in Warsaw, Poland.

WHAT: Emmanuel Macron assured the Polish leadership that France was not pro-Russian, but rather pro-European. He hoped the Polish people would trust Europe to protect them if there were any attacks. Macron emphasized the need for European nations to stand firmly against Russia. This message was delivered during his first visit to the country since assuming the presidency. The French president's trip aimed to strengthen relations between France and Poland and reinforce the message that Europe, not Russia, is the trusted ally. Macron's visit follows a pattern of European leaders seeking to solidify their cont

WHO: The main actors in the article are the presidential candidates Małgorzata Kidawa-Błońska (PO), Andrzej Duda, Władysław Kosiniak-Kamysz (PSL), Robert Biedroń, Szymon Hołownia and Krzysztof Bosak.

WHEN: The article focuses on rallies held on Saturday and Sunday as part of the presidential campaigns. 

WHERE: The rallies took place in Poland, in cities including Rzeszów.

WHAT: On Saturday and Sunday, several opposition candidates held rallies ahead of the Polish presidential election. Małgorzata Kidawa-Błońska (PO), currently the main challenger to incumbent Andrzej Duda, promised to reverse PiS reforms and advocated for women's rights, among other things. At her rally, Kidawa-Błońska highlighted the importance of the rule of law and called for Poland's first female president.

Władysław Kosiniak-Kamysz (PSL), another leading contender, positioned himself as a hopeful alternative to Duda, emphasising his commitment to the constitution and improving Poland's healthcare system. Rober

WHO: Hildur Guðnadóttir is the primary actor mentioned in the article, as it reviews the 'Joker: Original Motion Picture Soundtrack' composed by her. Andy Whitaker, the author of the article, is a secondary actor.

WHEN: The article was written and the soundtrack was released in December 2019.

WHERE: No specific location is mentioned in the article. However, the film and soundtrack are being discussed in relation to the Joker, which is set in Gotham City.

WHAT: The article summarises and reviews the 'Joker: Original Motion Picture Soundtrack'. It highlights the soundtrack's classical orchestration, featuring the cello as the leading instrument, and notes the short duration of the tracks. While the reviewer, Andy Whitaker, enjoys the score's sombre, melancholy and ominous tone, they remark that listening to the soundtrack prompts them to want to watch the film. Whitaker also appreciates the musical score's ability to evoke emotions and create a sense of anticipation. The article concl

WHO: The main actor in this news article is the South Dakota Department of Health (DOH). The article also highlights the cases of several counties in South Dakota, namely Lincoln, Bon Homme, Yankton, Clay, Union and Hutchinson. 

WHEN: The article focuses on the situation on April 12th, when the state of South Dakota had a total of 730 recorded cases of COVID-19. 

WHERE: The article exclusively discusses the situation in South Dakota. Several counties within the state are mentioned, as well as the state itself.

WHAT: The South Dakota Department of Health announced a rise of 104 new cases of COVID-19 on April 12th, increasing the state count to a total of 730 cases. Lincoln County reported 7 new cases, Bon Homme County 1, and Yankton County 20. The article also breaks down the cases by demographic, highlighting the number of hospitalizations, and deaths, and recovery rates by county and gender. It concludes with a list of the counties in South Dakota that have at least one case of COV

WHO: The main actor in this news article is the Federation of Associations of Private School Administrators (FAPSA), with its president Eleazardo Kasilag making the group's sentiments very clear. The secondary actors are the government bodies responsible for education policy, including the Inter-Agency Task Force for the Management of Emerging Infectious Diseases (IATF-MEID) and the Department of Education, who FAPSA are petitioning for help. 

WHEN: The article appears to be reporting on a current event, with the main action taking place on a single day – likely the day of publication. The specific timeline referred to in the article spans from March, when classes were stopped, to September, the proposed resumption of classes. 

WHERE: The article does not explicitly say where FAPSA is based, but it does mention that the events are relevant to the Philippines, as it refers to the DSWD (Department of Social Welfare and Development) of that country. 

WHAT: The FAPSA is expressing disma

### Polish language summarization

#### Without documents

In [330]:
def make_prompt_polish(news_article):
    return f'''## Instrukcje

    Podsumuj artykuł prasowy, koncentrując się na czterech głównych punktach: 
    1. KTO jest głównym/podrzędnym aktorem w artykule? 
    2. KIEDY miała miejsce sytuacja opisana w artykule? 
    3. GDZIE miała miejsce sytuacja opisana w artykule? 
    4. CO wydarzyło się w artykule (200-250 słów podsumowania artykułu)?
    
    Zwróć wynik w języku polskim.

    ## Przykładowe dane wyjściowe
    KTO: <odpowiedź na 1 punkt instrukcji>
    KIEDY: <odpowiedź na 2 punkt instrukcji>
    GDZIE: <odpowiedź na 3 punkt instrukcji>
    CO: <odpowiedź na 4 punkt instrukcji (200-250 słów podsumowania artykułu prasowego)>.

    ## Artykuł

    {news_article}
    '''

In [335]:
mixed_first_part_pl = ukr_1[ukr_1.url2_lang == 'pl'].copy()
mixed_first_part_pl.loc[:, "prompt_summ_text2"] = mixed_first_part_pl.loc[:, "text2"].apply(make_prompt_polish).values
mixed_first_part_pl.loc[:, "summ_text2"] = mixed_first_part_pl.loc[:, "prompt_summ_text2"].apply(query_command_r).values
mixed_second_part_pl = ukr_2[ukr_2.url1_lang == 'pl'].copy()
mixed_second_part_pl.loc[:, "prompt_summ_text1"] = mixed_second_part_pl.loc[:, "text1"].apply(make_prompt_polish).values
mixed_second_part_pl.loc[:, "summ_text1"] = mixed_second_part_pl.loc[:, "prompt_summ_text1"].apply(query_command_r).values

KTO: Komisja Europejska (KE), jej przewodnicząca Ursula von der Leyen oraz przewodniczący Rady Europejskiej Charles Michel.

KIEDY: Propozycja została przedstawiona w środę, 27 maja 2020 roku.

GDZIE: W Brukseli, gdzie znajduje się siedziba Komisji Europejskiej.

CO: KE zaproponowała budżet UE na lata 2021-2027 w wysokości 1,1 bln euro, dodatkowo państwa członkowskie miałyby otrzymać 750 mld euro w ramach instrumentu ożywienia gospodarczego. Polska miałaby otrzymać ok. 64 mld euro w ramach tego instrumentu. KE proponuje nowe źródła finansowania, m.in. opłaty za emisję CO2, podatek cyfrowy czy od dużych firm. Dodatkowo KE chce zwiększyć Fundusz Sprawiedliwej Transformacji do 40 mld euro, by przyspieszyć osiągnięcie przez państwa członkowskie neutralności klimatycznej.

KE proponuje też zwiększenie budżetu na politykę spójności o 55 mld euro, który zostałby przydzielony w zależności od skutków kryzysu dla danego państwa. Dodatkowo KE chce powołać nowy program zdrowotny EU4Health o budżec

KTO: Leszek Miller jest głównym aktorem artykułu, który komentuje sytuację polityczną w Polsce w kontekście wyborów prezydenckich 2020.

KIEDY: Wybory prezydenckie 2020 oraz nieokreślona przeszłość ("dawne czasy"), kiedy Miller pełnił funkcję premiera.

GDZIE: Polska.

CO: Leszek Miller wyraża opinię, że kampania wyborcza prezydencka 2020 nie będzie mieć wielkiego znaczenia, ponieważ nie jest starcie gigantów, jak miało to miejsce w poprzednich wyborach. Zwraca uwagę na brak atmosfery wielkiego wydarzenia oraz brak atrakcyjnych kandydatur, co jego zdaniem może wpłynąć na niską frekwencję. Miller krytykuje wszystkich kandydatów na prezydenta za składanie obietnic wykraczających poza kompetencje tego urzędu oraz proponuje wybór prezydenta przez Zgromadzenie Narodowe. Komentuje również zachowanie sędziego Macieja Nawackiego, którego gest podarcia apelacji określa jako nie do zniesienia. Następnie koncentruje się na podpisaniu przez Andrzeja Dudę kontrowersyjnej ustawy dyscyplinującej sędz

KTO: Pielęgniarka z powiatu szydłowieckiego jest głównym bohaterem artykułu.

KIEDY: Artykuł opisuje wydarzenia, które miały miejsce 16 i 17 marca.

GDZIE: Akcja artykułu rozgrywa się w Szpitalu Powiatowym w Skarżysku-Kamiennej, gdzie pracowała pielęgniarka, oraz w szpitalu w Grójcu, gdzie się zakażała.

CO: U pielęgniarki zatrudnionej w Szpitalu Powiatowym w Skarżysku-Kamiennej wykryto zakażenie koronawirusem. Od 48 pracowników i 17 pacjentów oddziału neurologicznego, na którym pracowała, pobrano wymazy do badań. Oddział został zamknięty do czasu uzyskania wyników i podjęcia dalszych decyzji. Okazało się, że pielęgniarka pracowała również w szpitalu w Grójcu, gdzie najprawdopodobniej doszło do zakażenia. Dyrektor skarżyskiej lecznicy podkreśla, że nie ma zakazów pracy w wielu szpitalach i zwraca uwagę na problemy kadrowe w placówkach medycznych. Autor artykułu apeluje, aby nie panikować i polegać na sprawdzonych źródłach informacji na temat koronawirusa.
KTO: Władze Bawarii, w osobach

KTO: Główne postacią artykułu jest USS Theodore Roosevelt - lotniskowiec marynarki wojennej USA, a także Thomas Modly - pełniący obowiązki sekretarza marynarki wojennej USA. 

KIEDY: Akcja artykułu ma miejsce w czasie pandemii COVID-19, która została ogłoszona przez Światową Organizację Zdrowia (WHO) 11 marca 2020 roku. 

GDZIE: Sytuacja opisana w artykule ma miejsce na Oceanie Spokojnym, gdzie pływa lotniskowiec USS Theodore Roosevelt, oraz w bazie marynarki wojennej USA na wyspie Guam, dokąd został skierowany okręt. 

CO: Lotniskowiec USS Theodore Roosevelt został skierowany do bazy na wyspie Guam z powodu wybuchu epidemii COVID-19 na jego pokładzie. Około tysiąca członków załogi zostało ewakuowanych, a w najbliższych dniach planowana jest ewakuacja kolejnych 2700 osób. Pełniący obowiązki sekretarza marynarki wojennej USA Thomas Modly podkreśla, że część załogi musi pozostawać na pokładzie, aby zapewnić funkcjonowanie systemów okrętu i jego ochronę. Decyzja o całkowitej ewakuacji zał

#### With documents

In [336]:
prompt_command_r_pl = '''
## Instrukcje

    Podsumuj artykuł prasowy z załączonych dokumentów, koncentrując się na czterech głównych punktach: 
    1. KTO jest głównym/podrzędnym aktorem w artykule prasowym? 
    2. KIEDY miała miejsce sytuacja opisana w artykule? 
    3. GDZIE miała miejsce sytuacja opisana w artykule? 
    4. CO wydarzyło się w artykule (200-250 słów podsumowania artykułu)?
    
Zwróć wynik w języku polskim.

## Przykładowe dane wyjściowe
    KTO: <odpowiedź na 1 punkt instrukcji>
    KIEDY: <odpowiedź na 2 punkt instrukcji>
    GDZIE: <odpowiedź na 3 punkt instrukcji>
    CO: <odpowiedź na 4 punkt instrukcji (200-250 słów podsumowania artykułu prasowego)>.

## Artykuł
    '''

In [337]:
mixed_first_part_pl.loc[:, "documents_text2"] = mixed_first_part_pl.loc[:, "text2"].apply(other_languages_generate_documents_command_r_version_1, language='polish')
mixed_second_part_pl.loc[:, "documents_text1"] = mixed_second_part_pl.loc[:, "text1"].apply(other_languages_generate_documents_command_r_version_1, language='polish')
mixed_first_part_pl.loc[:, "summ_with_documents_text2"] = mixed_first_part_pl.loc[:, "documents_text2"].apply(query_command_r_documents, args=(prompt_command_r_pl, ))
mixed_second_part_pl.loc[:, "summ_with_documents_text1"] = mixed_second_part_pl.loc[:, "documents_text1"].apply(query_command_r_documents, args=(prompt_command_r_pl, ))

KTO: Komisja Europejska (KE), przewodnicząca KE Ursula von der Leyen, państwa członkowskie UE

KIEDY: Propozycje KE przedstawiono w środę, czyli w czasie pandemii Covid-19. Szczyt UE, podczas którego szefowie państw i rządów zajęli się propozycjami KE, odbył się 19 czerwca.

GDZIE: Propozycje przedstawiono w Parlamencie Europejskim, a zajęto się nimi podczas szczytu UE w Brukseli.

CO: KE zaproponowała nowy, 750 mld euro mechanizm obudowy gospodarczej po pandemii koronawirusa, który ma wesprzeć odbudowę jednolitego rynku UE i przywrócić równe warunki funkcjonowania na nim wszystkich przedsiębiorstw. Pieniądze mają pochodzić z podwyższenia pułapu zasobów własnych UE, a KE będzie je pożyczać z rynków finansowych, wykorzystując swój wysoki rating kredytowy. Z przeznaczone na odbudowę środki, 250 mld euro mają być pożyczkami, a 500 mld euro grantami dla krajów unijnych. KE proponuje też zwiększyć budżet UE na lata 2021-2027 do 1,1 biliona euro i uzupełnić obecne programy polityki spójności

KTO: Głównym aktorem artykułu jest prezydent Andrzej Duda, który był gościem programu „Rozmowy niedokończone” Telewizji Trwam i Radia Maryja. W trakcie wywiadu, do studia zadzwonił redemptorysta o. Tadeusz Rydzyk.

KIEDY: Sytuacja opisana w artykule miała miejsce w przeszłości, jednak dokładna data nie została podana.

GDZIE: Wydarzenia opisane w artykule miały miejsce w studio Telewizji Trwam i Radia Maryja, czyli w miejscu zlokalizowanym prawdopodobnie w Polsce.

CO: Andrzej Duda był gościem programu „Rozmowy niedokończone”. W trakcie wywiadu, prezydent mówił m.in. o koronawirusie i związanych z nim problemach gospodarczych, wierząc, że Polsce uda się przez nie przejść i powrócić na drogę szybkiego rozwoju. Apelował do Polaków o wsparcie i wyrozumiałość, oraz zachęcał do udziału w wyborach. 
Do studia zadzwonił o. Rydzyk, który podziękował Dudzie za jego siłę oraz zwrócił uwagę na problemy w polskich szkołach, związane z obecnością "genderowych historii". Ponadto założyciel Radia Mar

KTO: Główne postacią artykułu jest pielęgniarka z powiatu szydłowieckiego, która pracowała w Szpitalu Powiatowym w Skarżysku-Kamiennej.

KIEDY: Sytuacja opisana w artykule miała miejsce 16 i 17 marca.

GDZIE: Akcja artykułu rozgrywa się w Szpitalu Powiatowym w Skarżysku-Kamiennej oraz w szpitalu w Grójcu koło Warszawy, gdzie również pracowała zakażona pielęgniarka.

CO: W artykule opisana jest sytuacja, która miała miejsce na początku epidemii koronawirusa w Polsce. U pielęgniarki, która pracowała w Szpitalu Powiatowym w Skarżysku-Kamiennej 16 i 17 marca, wykryto zakażenie koronawirusem. W wyniku tego zamknięty został cały oddział, na którym pracowała, a osoby, które z nią miały kontakt, zostały poddane kwarantannie i czekały na wyniki testów. Dyrektor szpitala podkreśla, że nie ma zakazów pracy w wielu placówkach jednocześnie, co w tym przypadku zaowocowało zakażeniem wielu osób. Pozostali pacjenci i personel szpitala nadal pracowali bez zmian.
KTO: Głównym aktorem w artykule prasowym

KTO: Głównym aktorem artykułu jest lotniskowiec USS Theodore Roosevelt, a podrzędnymi aktorami pełniący obowiązki sekretarza marynarki wojennej USA Thomas Modly oraz kapitan lotniskowca. 

KIEDY: Sytuacja opisana w artykule miała miejsce w czasie pandemii koronawirusa COVID-19, kiedy to lotniskowiec USS Theodore Roosevelt pełnił zadania na Oceanie Spokojnym. 

GDZIE: Akcja artykułu rozgrywa się na lotniskowcu USS Theodore Roosevelt, który cumuje w bazie na wyspie Guam. 

CO: W artykule opisana jest sytuacja związana z ewakuacją części załogi lotniskowca USS Theodore Roosevelt, która miała miejsce w czasie pandemii COVID-19. Kapitan okrętu zwrócił się z prośbą o ewakuację 4 tysięcy osób na ląd, jednak jak oświadczył pełniący obowiązki sekretarza marynarki wojennej USA, Thomas Modly, nie ma możliwości ewakuacji całej załogi. Prioritetem jest ewakuacja osób z objawami choroby. Według danych podanych przez kapitana, na pokładzie okrętu znajduje się wielu zarażonych koronawirusem — zarówno 

### Final deduplication

In [338]:
dataset = pd.concat([ukr_ukr, mixed_second_part_pl, mixed_second_part_en, mixed_second_part_ru, mixed_first_part_pl, mixed_first_part_en, mixed_first_part_ru], ignore_index=True)
dataset = dataset.drop_duplicates(subset=["source1", "source2"], ignore_index=True)
dataset = dataset.drop_duplicates(subset=["text1", "text2"], ignore_index=True)
dataset = dataset[dataset['source1'] != dataset['source2']]
dataset = dataset[dataset['text1'] != dataset['text2']]

In [11]:
def delete_citations(summarization):
    try:
        pattern = r"<co: [\d,]+>|</co: [\d,]+>"
        return re.sub(pattern, "", summarization)
    except Exception as e:
        print(e)
    return summarization

In [340]:
dataset["summ_with_documents_text1"] = dataset["summ_with_documents_text1"].apply(delete_citations)
dataset["summ_with_documents_text2"] = dataset["summ_with_documents_text2"].apply(delete_citations)

### Toloka format

In [343]:
for_Toloka = dataset.add_prefix('INPUT:').copy()

In [344]:
for_Toloka_ukr_ukr = for_Toloka[(for_Toloka['INPUT:url1_lang'] == 'uk') & (for_Toloka['INPUT:url2_lang'] == 'uk')]
for_Toloka_ukr_pl = for_Toloka[((for_Toloka['INPUT:url1_lang'] == 'pl') & (for_Toloka['INPUT:url2_lang'] == 'uk')) | (for_Toloka['INPUT:url1_lang'] == 'uk') & (for_Toloka['INPUT:url2_lang'] == 'pl')]
for_Toloka_ukr_ru = for_Toloka[((for_Toloka['INPUT:url1_lang'] == 'ru') & (for_Toloka['INPUT:url2_lang'] == 'uk')) | (for_Toloka['INPUT:url1_lang'] == 'uk') & (for_Toloka['INPUT:url2_lang'] == 'ru')]
for_Toloka_ukr_en = for_Toloka[((for_Toloka['INPUT:url1_lang'] == 'en') & (for_Toloka['INPUT:url2_lang'] == 'uk')) | (for_Toloka['INPUT:url1_lang'] == 'uk') & (for_Toloka['INPUT:url2_lang'] == 'en')]

In [349]:
for_Toloka_ukr_ukr = for_Toloka_ukr_ukr[['INPUT:source2', 'INPUT:text2',
       'INPUT:source1', 'INPUT:text1', 'INPUT:summ_text1', 'INPUT:summ_text2', 
       'INPUT:summ_with_documents_text1', 'INPUT:summ_with_documents_text2']]
for_Toloka_ukr_ru = for_Toloka_ukr_ru[['INPUT:source2', 'INPUT:text2',
       'INPUT:source1', 'INPUT:text1', 'INPUT:summ_text1', 'INPUT:summ_text2', 
       'INPUT:summ_with_documents_text1', 'INPUT:summ_with_documents_text2']]
for_Toloka_ukr_pl = for_Toloka_ukr_pl[['INPUT:source2', 'INPUT:text2',
       'INPUT:source1', 'INPUT:text1', 'INPUT:summ_text1', 'INPUT:summ_text2', 
       'INPUT:summ_with_documents_text1', 'INPUT:summ_with_documents_text2']]
for_Toloka_ukr_en = for_Toloka_ukr_en[['INPUT:source2', 'INPUT:text2',
       'INPUT:source1', 'INPUT:text1', 'INPUT:summ_text1', 'INPUT:summ_text2', 
       'INPUT:summ_with_documents_text1', 'INPUT:summ_with_documents_text2']]